<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

## Phonemizer

Assumes espeak backend is installed via `apt-get install espeak`

In [1]:
#| echo: false
#| output: asis
show_doc(Phonemizer)

---

[source](https://github.com/slegroux/nimrod/blob/main/nimrod/text/tokenizers.py#L25){target="_blank" style="float:right; font-size:smaller"}

### Phonemizer

>      Phonemizer (separator=<phonemizer.separator.Separator object at
>                  0x7fdf1c873070>, language='en-us', backend='espeak',
>                  strip=True, preserve_punctuation=True)

Initialize self.  See help(type(self)) for accurate signature.

### Usage

In [ ]:
p = Phonemizer()
text = "oh shoot I missed my train"
print(p(text))
text = ["Oh Dear, you'll be fine!", "this is it"]
print(p(text))

oʊ ʃuːt aɪ mɪst maɪ tɹeɪn
['oʊ dɪɹ, juːl biː faɪn!', 'ðɪs ɪz ɪt']


### Lhotse tokenization

In [ ]:
phon = Phonemizer()
cuts = CutSet.from_file("../data/en/LJSpeech-1.1/first_3.jsonl.gz")
# cuts = CutSet.from_file("../recipes/tts/ljspeech/data/ljspeech_fbank_all.jsonl.gz")
unique_phonemes = set()
print(cuts[0])
with CutSet.open_writer('../data/en/LJSpeech-1.1/first_3.phon.jsonl.gz', overwrite=True) as writer:
    for cut in tqdm(cuts):
        phonemes = phon(cut.supervisions[0].text, n_jobs=1)
        cut.custom = {"phonemes": phonemes}
        writer.write(cut, flush=True)
        unique_phonemes.update(list(phonemes))
print(unique_phonemes, len(unique_phonemes))

MonoCut(id='LJ001-0001-0', start=0, duration=9.65501133786848, channel=0, supervisions=[SupervisionSegment(id='LJ001-0001', recording_id='LJ001-0001', start=0.0, duration=9.65501133786848, channel=0, text='Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition', language='English', speaker=None, gender='female', custom=None, alignment=None)], features=None, recording=Recording(id='LJ001-0001', sources=[AudioSource(type='file', channels=[0], source='/data/en/LJSpeech/LJSpeech-1.1/wavs/LJ001-0001.wav')], sampling_rate=22050, num_samples=212893, duration=9.65501133786848, channel_ids=[0], transforms=None), custom=None)


  0%|          | 0/3 [00:00<?, ?it/s]

{'ɪ', 'e', 'ɔ', 'ɑ', 'ə', 'ɛ', 'p', 'o', 'ŋ', 'z', 'a', 'ɾ', 'b', 'ð', 'd', 's', 'm', 'v', 'l', 'k', 'ᵻ', 'ʊ', 'i', ' ', 'ɡ', 'ː', 'n', 't', 'w', 'ɜ', 'ɚ', 'ʃ', 'f', 'ɹ', ',', 'ɐ', 'æ', 'ʌ', '.'} 39


In [ ]:
processed_cuts = CutSet.from_file("../data/en/LJSpeech-1.1/first_3.phon.jsonl.gz")
print(processed_cuts[0])

MonoCut(id='LJ001-0001-0', start=0, duration=9.65501133786848, channel=0, supervisions=[SupervisionSegment(id='LJ001-0001', recording_id='LJ001-0001', start=0.0, duration=9.65501133786848, channel=0, text='Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition', language='English', speaker=None, gender='female', custom=None, alignment=None)], features=None, recording=Recording(id='LJ001-0001', sources=[AudioSource(type='file', channels=[0], source='/data/en/LJSpeech/LJSpeech-1.1/wavs/LJ001-0001.wav')], sampling_rate=22050, num_samples=212893, duration=9.65501133786848, channel_ids=[0], transforms=None), custom={'phonemes': 'pɹɪntɪŋ, ɪnðɪ oʊnli sɛns wɪð wɪtʃ wiː ɑːɹ æt pɹɛzənt kənsɜːnd, dɪfɚz fɹʌm moʊst ɪf nɑːt fɹʌm ɔːl ðɪ ɑːɹts ænd kɹæfts ɹɛpɹᵻzɛntᵻd ɪnðɪ ɛksɪbɪʃən'})


In [ ]:
map = {}
unique_syms = set()
for cut in processed_cuts:
    unique_syms.update(list(cut.custom['phonemes']))

for (i, v) in enumerate(sorted(list(unique_syms))):
    map[i] = v
print(map, len(map))

{0: ' ', 1: ',', 2: '.', 3: 'a', 4: 'b', 5: 'd', 6: 'e', 7: 'f', 8: 'i', 9: 'k', 10: 'l', 11: 'm', 12: 'n', 13: 'o', 14: 'p', 15: 's', 16: 't', 17: 'v', 18: 'w', 19: 'z', 20: 'æ', 21: 'ð', 22: 'ŋ', 23: 'ɐ', 24: 'ɑ', 25: 'ɔ', 26: 'ə', 27: 'ɚ', 28: 'ɛ', 29: 'ɜ', 30: 'ɡ', 31: 'ɪ', 32: 'ɹ', 33: 'ɾ', 34: 'ʃ', 35: 'ʊ', 36: 'ʌ', 37: 'ː', 38: 'ᵻ'} 39


## Tokenizer

Requires download of spacy specific language e.g. `python -m spacy download en`

In [2]:
#| echo: false
#| output: asis
show_doc(Tokenizer)

---

[source](https://github.com/slegroux/nimrod/blob/main/nimrod/text/tokenizers.py#L79){target="_blank" style="float:right; font-size:smaller"}

### Tokenizer

>      Tokenizer (backend='spacy', language='en')

Initialize self.  See help(type(self)) for accurate signature.

## Usage

In [ ]:
tok = Tokenizer()

In [ ]:
# str -> List[str]
s = "Oh, yeah I don't know dude..."
tokenized = tok(s)
print(s)
print(tokenized)
print(tok.inverse(tokenized))

# List[str]->List[List[str]]
s = ["Oh, yeah I don't know dude...", "this is a test"]
tokenized = tok(s)
print(tokenized)
print(tok.inverse(tokenized))

# Iterable -> Iterable
ds = AG_NEWS(split='test') # data pipe
sample = next(iter(ds)) # (label, text)
# print(sample)
it = tok(ds)
tokens = [token for token in it]
print(tokens[:2])

Oh, yeah I don't know dude...
['Oh', ',', 'yeah', 'I', 'do', "n't", 'know', 'dude', '...']
Oh , yeah I do n't know dude ...
[['Oh', ',', 'yeah', 'I', 'do', "n't", 'know', 'dude', '...'], ['this', 'is', 'a', 'test']]
["Oh , yeah I do n't know dude ...", 'this is a test']
[['Fears', 'for', 'T', 'N', 'pension', 'after', 'talks', 'Unions', 'representing', 'workers', 'at', 'Turner', '  ', 'Newall', 'say', 'they', 'are', "'", 'disappointed', "'", 'after', 'talks', 'with', 'stricken', 'parent', 'firm', 'Federal', 'Mogul', '.'], ['The', 'Race', 'is', 'On', ':', 'Second', 'Private', 'Team', 'Sets', 'Launch', 'Date', 'for', 'Human', 'Spaceflight', '(', 'SPACE.com', ')', 'SPACE.com', '-', 'TORONTO', ',', 'Canada', '--', 'A', 'second\\team', 'of', 'rocketeers', 'competing', 'for', 'the', ' ', '#', '36;10', 'million', 'Ansari', 'X', 'Prize', ',', 'a', 'contest', 'for\\privately', 'funded', 'suborbital', 'space', 'flight', ',', 'has', 'officially', 'announced', 'the', 'first\\launch', 'date', 'for',

## Numericalizer

In [3]:
#| echo: false
#| output: asis
show_doc(Numericalizer)

---

[source](https://github.com/slegroux/nimrod/blob/main/nimrod/text/tokenizers.py#L113){target="_blank" style="float:right; font-size:smaller"}

### Numericalizer

>      Numericalizer (tokens_iter:Iterable, specials=['<pad>', '<unk>', '<bos>',
>                     '<eos>'])

Initialize self.  See help(type(self)) for accurate signature.

## Usage

In [ ]:
tok = Tokenizer()
# In the case of agnews, dataset is: [(index, text)]
def token_iterator(data_iter:Iterable)->Iterable:
    for _, text in data_iter:
        yield tok(text)
tok_it= token_iterator(ds)
# initialize numericalizer based on token iterator
num = Numericalizer(tok_it)

In [ ]:
print(num('<pad>'), num('<unk>'))

0 1


In [ ]:
print(num.vocab['the'])
print(num('the'))
print(num(['<bos>', '<pad>', '<unk>', 'a', 'this', 'the', 'lkjsdf']))
print(num.inverse(0))
print(num.inverse([6,55]))
print(num([['<bos>', '<pad>'], ['<unk>', 'a', 'this', 'the', 'lkjsdf']]))

4
4
[2, 0, 1, 9, 58, 4, 1]
<pad>
['.', 'Monday']
[[2, 0], [1, 9, 58, 4, 1]]


In [ ]:
tokens = tok(["here we go. asdflkj", "it was time..."])
print(tokens)
print([num(tok) for tok in tokens])
print(num(tokens))

[['here', 'we', 'go', '.', 'asdflkj'], ['it', 'was', 'time', '...']]
[[534, 1040, 310, 6, 1], [34, 40, 101, 67]]
[[534, 1040, 310, 6, 1], [34, 40, 101, 67]]


In [4]:
#| echo: false
#| output: asis
show_doc(TextCollater)

---

[source](https://github.com/slegroux/nimrod/blob/main/nimrod/text/tokenizers.py#L154){target="_blank" style="float:right; font-size:smaller"}

### TextCollater

>      TextCollater (tokenizer, numericalizer, padding_value:int=0)

Initialize self.  See help(type(self)) for accurate signature.

In [ ]:
texts = ["this is it...", "this is the second sentence."]
t = tok(texts)
print(t)
tt = num(t)
print(tt)
ttt= [torch.Tensor(t) for t in tt]
[t.shape[0] for t in ttt]
collater = TextCollater(tok, num)
print(collater.collate_list(texts))
dl = DataLoader(dataset=ds, batch_size=2, shuffle=True, collate_fn=collater.collate_agnews)

[['this', 'is', 'it', '...'], ['this', 'is', 'the', 'second', 'sentence', '.']]
[[58, 27, 34, 67], [58, 27, 4, 95, 3714, 6]]
(tensor([[  58,   27,   34,   67,    0,    0],
        [  58,   27,    4,   95, 3714,    6]]), tensor([4, 6]))


In [ ]:
b = next(iter(dl))
print('batch: ', b)
tokens, lens = b[0], b[1]
for token, len in zip(tokens, lens):
    print(token[:len].tolist())
    print(num.inverse(token[:len].tolist()))

batch:  (tensor([[ 3894,  6448,    13,   532,   179,  9855,  1683,    14,   932,    17,
          3586,    18,   453,    11,  3894,   371,  2260,     5,     4,   100,
            16,    19,   171,   284,     8,   312,  1820,     5,  1406,    42,
            22,   454,   401,  2514,    28,     4,    63,    27,  2297,  3327,
            21,    41,  2455,  2275,   797,    69],
        [ 3592,  6422,  3288,     7,  1359,   928,    15,  2846,    17,    36,
            18,     8,   212,   119,  2290,    71,   157,    15,  3202,  1163,
          8066,    60,    13,  7023,   971,    20,    15,  1673,    10,  2735,
          6044,    39,  2951, 10519,   194,     4,    15,   873,  8398,  1748,
             6,     0,     0,     0,     0,     0]]), tensor([46, 41]))
[3894, 6448, 13, 532, 179, 9855, 1683, 14, 932, 17, 3586, 18, 453, 11, 3894, 371, 2260, 5, 4, 100, 16, 19, 171, 284, 8, 312, 1820, 5, 1406, 42, 22, 454, 401, 2514, 28, 4, 63, 27, 2297, 3327, 21, 41, 2455, 2275, 797, 69]
['Vodafone', 'D